# Тестовое задание на позицию "Бизнес-аналитик" (SQL).

-- ТАБЛИЦА С ДОГОВОРАМИ
```
select 
    DWH_PERSON_ID, -- id клиента
    DWH_DEBT_ID, -- номер договора
    CREDIT_START_DATE, -- дата выдачи кредита
    BIRTH_DATE, -- дата рождения
    CITY_NAME, -- город выдачи кредита
    CREDIT_SUM – сумма выданного кредита
from u_collection.dwh_debt
```
---------------------------------------------------
--ТАБЛИЦА С КЛИЕНТАМИ
```
select 
    DWH_PERSON_ID, -- id клиента
    sex -- пол клиента (м, ж)
from u_collection.dwh_person;
````
---------------------------------------------------
--ТАБЛИЦА С ОБЕЩАНИЯМИ ВНЕСТИ ПЛАТЕЖ (их может быть много по каждому договору)
```
select 
    DWH_DEBT_ID, -- номер договора
    PROM_DATE, -- дата обещания
    PROM_SUM, -- обещанная сумма
    COVER_SUM, -- сумма платежа, внесенная в рамках обещания
from u_collection.dwh_person;
```

	
Необходимо выбрать все номера договоров, которые выданы клиентам женского пола в возрасте от 25 -35 полных лет, при условии, что:

1.	дата выдачи больше 1 января 2014 года и сумма кредита больше средней суммы всех выданных кредитов 
2.	третья буква в городе выдачи не равна "о"
3.	последняя дата обещания по договору прошла более 10 дней назад
4.	сумма внесенных в рамках обещаний платежей с даты выдачи кредита >= 20000 р. 





# SQL LITE

In [ ]:
# Импорт библиотек
import sqlite3
sqlite3.sqlite_version

In [ ]:
# Подключение к файлу базы данных с именем “u_collection.db”
con1 = sqlite3.connect('u_collection.db')

cursor = con1.cursor()

# Создаем таблицу u_collection.dwh_debt
cursor.execute('''
CREATE TABLE IF NOT EXISTS dwh_debt (DWH_PERSON_ID INTEGER,
                                     DWH_DEBT_ID INTEGER PRIMARY KEY,
                                     CREDIT_START_DATE DATE,
                                     BIRTH_DATE DATE,
                                     CITY_NAME TEXT,
                                     CREDIT_SUM REAL)
''')  
# Создаем таблицу u_collection.dwh_person
cursor.execute('''                              
CREATE TABLE IF NOT EXISTS dwh_person (DWH_PERSON_ID INTEGER,
                                       sex TEXT,
                                       DWH_DEBT_ID INTEGER,
                                       PROM_DATE DATE,
                                       PROM_SUM REAL,
                                       COVER_SUM REAL)                              
''')
# Добавляем данные в таблицу dwh_debt
cursor.execute('''
INSERT OR IGNORE INTO dwh_debt (DWH_PERSON_ID,
                                DWH_DEBT_ID,
                                CREDIT_START_DATE,
                                BIRTH_DATE,
                                CITY_NAME,
                                CREDIT_SUM) 
VALUES 
(101, 1, '2013-01-15', '1990-05-20', 'Москва', 50000),
(102, 2, '2019-03-10', '1985-07-15', 'Санкт-Петербург', 30000),
(103, 3, '2021-06-25', '1991-11-10', 'Екатеринбург', 100000),
(104, 4, '2020-06-25', '1990-11-30', 'Бронницы', 80000),
(101, 5, '2021-03-25', '1992-01-20', 'Москва', 70000),
(105, 6, '2020-06-15', '1992-11-30', 'Екатеринбург', 50000),
(102, 7, '2022-07-21', '1991-01-20', 'Ярославль', 90000),
(106, 8, '2021-08-25', '1992-11-30', 'Москва', 70000),
(107, 9, '2020-09-25', '1990-11-20', 'Екатеринбург', 20000),
(108, 10, '2023-09-01', '1988-02-10', 'Казань', 45000)
''')
# Добавляем данные в таблицу dwh_person
cursor.execute('''
INSERT OR IGNORE INTO dwh_person (DWH_PERSON_ID,
                                  sex,
                                  DWH_DEBT_ID,
                                  PROM_DATE,
                                  PROM_SUM,
                                  COVER_SUM) 
VALUES 
(101, 'ж', 1, '2014-02-01', 10000, 5000),
(101, 'ж', 1, '2014-03-01', 5000, 5000),
(102, 'м', 2, '2019-04-01', 15000, 15000),
(103, 'ж', 3, '2021-07-01', 20000, 10000),
(104, 'м', 4, '2020-08-25', 10000, 8000),
(104, 'м', 4, '2020-10-25', 2000, 1000),
(101, 'ж', 5, '2021-05-01', 20000, 19000),
(105, 'ж', 6, '2020-08-01', 10000, 10000),
(102, 'ж', 7, '2022-09-08', 20000, 10000),
(106, 'м', 8, '2021-10-01', 15000, 10000),
(106, 'м', 8, '2021-11-01', 5000, 3000),
(107, 'ж', 9, '2020-10-05', 20000, 10000),
(108, 'ж', 10, '2023-11-01', 5000, 5000)
''')
# Сохраняем изменения и закрываем соединение
con1.commit()
con1.close()

In [ ]:
# Устанавливаем соединение с базой данных
con2 = sqlite3.connect('u_collection.db')
cursor = con2.cursor()

# Выполняем запрос
cursor.execute('''
WITH
-- ТАБЛИЦА С ДОГОВОРАМИ
orders AS (
SELECT DWH_PERSON_ID,
       DWH_DEBT_ID,
       CREDIT_START_DATE,
       BIRTH_DATE,
       CITY_NAME,
       CREDIT_SUM
FROM dwh_debt
),
--ТАБЛИЦА С КЛИЕНТАМИ 
person_sex AS (
SELECT DWH_PERSON_ID,
       sex
FROM dwh_person
),
--ТАБЛИЦА С ОБЕЩАНИЯМИ ВНЕСТИ ПЛАТЕЖ (их может быть много по каждому договору)
payment_promise AS (
SELECT DWH_DEBT_ID,
       PROM_DATE,
       PROM_SUM,
       COVER_SUM
FROM dwh_person
),
--Запрос для получения среднего значения всех выднных кредитов
average_credit AS (
SELECT AVG(CREDIT_SUM) AS avg_credit
FROM orders
)

SELECT *
FROM orders

''')
res_1 = cursor.fetchall()

# Выводим результаты
for s in res_1:
    print(*s)
# Закрываем соединение
con2.close()

**Замечание:** 
Далее принято решение продолжить выполнение задания используя POSTGRE SQL.  
Синтаксис SQL LITE довольно сильно отличается от POSTGRE.   

# POSTGRE SQL (проверенно на сайте https://onecompiler.com/postgresql/42ttksfmq)

**Описание решения.**  
В данной задаче я попытался сымитировать имеющуюся базу данных.  
Из условия задачи предположил что в базе данных `u_collection` имеется 2 таблицы: `dwh_debt` и `dwh_person`.  
Далее из этих таблиц с помощью простых запросов получается 3 таблицы (из таблицы `u_collection.dwh_person` с помощью запросов получается 2 таблицы), используя данные которых необходимо выполнить целевой запрос.  

**Первый этап:** Создаём 2 исходные таблицы `dwh_debt` и `dwh_person`и наполняем их игрушечными данными.

In [ ]:
-- Создание таблицы dwh_debt
CREATE TABLE dwh_debt (DWH_PERSON_ID INTEGER,
                       DWH_DEBT_ID INTEGER PRIMARY KEY,
                       CREDIT_START_DATE DATE,
                       BIRTH_DATE DATE,
                       CITY_NAME TEXT,
                       CREDIT_SUM REAL)
;
-- Создание таблицы dwh_person
CREATE TABLE dwh_person (DWH_PERSON_ID INTEGER,
                         sex TEXT,
                         DWH_DEBT_ID INTEGER,
                         PROM_DATE DATE,
                         PROM_SUM REAL,
                         COVER_SUM REAL)
;
-- Добавление данных в таблицу dwh_debt
INSERT INTO dwh_debt (DWH_PERSON_ID,
                      DWH_DEBT_ID,
                      CREDIT_START_DATE,
                      BIRTH_DATE,
                      CITY_NAME,
                      CREDIT_SUM) 
VALUES 
(101, 1, '2013-01-15', '1990-05-20', 'Москва', 50000),
(102, 2, '2019-03-10', '1985-07-15', 'Санкт-Петербург', 30000),
(103, 3, '2021-06-25', '1991-11-10', 'Екатеринбург', 100000),
(104, 4, '2020-06-25', '1990-11-30', 'Бронницы', 80000),
(101, 5, '2021-03-25', '1992-01-20', 'Москва', 70000),
(105, 6, '2020-06-15', '1992-11-30', 'Екатеринбург', 50000),
(102, 7, '2022-07-21', '1991-01-20', 'Ярославль', 90000),
(106, 8, '2021-08-25', '1992-11-30', 'Москва', 70000),
(107, 9, '2020-09-25', '1990-11-20', 'Екатеринбург', 20000),
(108, 10, '2023-09-01', '1988-02-10', 'Казань', 45000)
;
-- Добавление данных в таблицу dwh_person
INSERT  INTO dwh_person (DWH_PERSON_ID,
                         sex,
                         DWH_DEBT_ID,
                         PROM_DATE,
                         PROM_SUM,
                         COVER_SUM) 
VALUES 
(101, 'ж', 1, '2014-02-01', 10000, 5000),
(101, 'ж', 1, '2014-03-01', 5000, 5000),
(102, 'м', 2, '2019-04-01', 15000, 15000),
(103, 'ж', 3, '2021-07-01', 20000, 10000),
(104, 'м', 4, '2020-08-25', 10000, 8000),
(104, 'м', 4, '2020-10-25', 2000, 1000),
(101, 'ж', 5, '2021-05-01', 25000, 21000),
(105, 'ж', 6, '2020-08-01', 10000, 10000),
(102, 'м', 7, '2022-09-08', 20000, 10000),
(106, 'м', 8, '2021-10-01', 15000, 10000),
(106, 'м', 8, '2021-11-01', 5000, 3000),
(107, 'ж', 9, '2020-10-05', 20000, 10000),
(108, 'ж', 10, '2023-11-01', 5000, 5000) 
;

**Второй этап:** С помощью табличных выражений создаем временные таблицы (исходные по условию задачи):
`orders` - ТАБЛИЦА С ДОГОВОРАМИ,
`person_sex` - ТАБЛИЦА С КЛИЕНТАМИ (здесь сразу для удобства удаляем дубликаты, иначе понадобится ещё один подзапрос),
`dwh_person` - ТАБЛИЦА С ОБЕЩАНИЯМИ ВНЕСТИ ПЛАТЕЖ.

In [ ]:
-- Запрос с использованием табличных выражений (CTE)
WITH
-- ТАБЛИЦА С ДОГОВОРАМИ
orders AS (
SELECT DWH_PERSON_ID,
       DWH_DEBT_ID,
       CREDIT_START_DATE,
       BIRTH_DATE,
       CITY_NAME,
       CREDIT_SUM
FROM dwh_debt
),
--ТАБЛИЦА С КЛИЕНТАМИ (данные пола без дубликатов)
person_sex AS (
SELECT DISTINCT DWH_PERSON_ID,
       sex
FROM dwh_person
),
--ТАБЛИЦА С ОБЕЩАНИЯМИ ВНЕСТИ ПЛАТЕЖ (их может быть много по каждому договору)
payment_promise AS (
SELECT DWH_DEBT_ID,
       PROM_DATE,
       PROM_SUM,
       COVER_SUM
FROM dwh_person
),

**Третий этап:** 
Выполняем требуемый по условию задачи запрос. Здесь также используем `CTE` (не пишем `WITH` так как это продолжение предыдущего запроса). 
1. Получаем среднее значение суммы всех кредитов.
2. Фильтруем данные по полу, возрасту , дате выдаче кредита и сумме кредита используя предыдущий подзапрос и названию города.
3. Далее в отдельном подзапросе сохраняем данные по обещаниям заемщиков (суммы платежей и последнюю дату платежа).
4. В основном запросе объединяем 2 полученные таблицы и фильтруем по последним двум условиям (последняя дата обещания по договору прошла более 10 дней назад и сумма внесенных в рамках обещаний платежей с даты выдачи кредита >= 20000 р.).

In [ ]:
--Подзапрос для получения среднего значения всех выднных кредитов

average_credit AS (
SELECT AVG(CREDIT_SUM) AS avg_credit
FROM orders
),

--Основной подзапрос, который выбирает необходимые данные применяя условия по полу, возрасту, дате выдачи кредита, сумме кредита и городу

filtered_debts AS (
SELECT DWH_DEBT_ID,
       CREDIT_START_DATE,
       CREDIT_SUM,
       BIRTH_DATE,
       CITY_NAME,
       sex
FROM orders
LEFT JOIN person_sex USING(DWH_PERSON_ID)
WHERE sex = 'ж'
AND DATE_PART('year', AGE(BIRTH_DATE)) BETWEEN 25 AND 35
AND CREDIT_START_DATE > '2014-01-01'
AND CREDIT_SUM > (SELECT avg_credit FROM average_credit)
AND SUBSTRING(CITY_NAME, 3, 1) <> 'о'
),

--Подзапрос для агрегирования данных по обещаниям, чтобы получить общую сумму внесенных платежей и последнюю дату обещания для каждого договора

filtered_promises AS (
SELECT DWH_DEBT_ID,
       SUM(COVER_SUM) AS total_covered_sum,
       MAX(PROM_DATE) AS last_prom_date
FROM payment_promise
GROUP BY DWH_DEBT_ID
)

-- Основной запрос
SELECT DWH_DEBT_ID
FROM filtered_debts
INNER JOIN filtered_promises USING(DWH_DEBT_ID)
WHERE last_prom_date < CURRENT_DATE - INTERVAL '10 days'
  AND total_covered_sum >= 20000


**Итог:** 
Работоспособность запроса была проверена на сайте `onecompiler.com`.